## Continuación Análisis de Negocios

## Índice

<a href = "#id1"> 1. Criminalidad desde el año 2015 hasta el 2018 <br>
<a href = "#id2"> 2. Graficación de las zonas en un mapa <br>
<a href = "#id3"> 3. Adición de la población de las zonas <br>
<a href = "#id3.1" style = "padding-left:1em"> 3.1. Ponderación de delitos y población <br>
<a href = "#id4"> 4. ¿Podemos sacar provecho a estos datos? <br>


## Criminalidad en los barrios de Phoenix - ¿Cuál es el barrio más seguro?
#### Adición de otros Dataset: Crimenes y población de Phoenix
Imaginamos un cliente que nos dice que necesita saber el barrio más seguro de Phoenix para poner un negocio, por lo que vamos a intentar obtenerlo.

El objetivo de esta parte es utilizar un dataset externo a los que nos habían proporcionado para concoer cuáles son los barrios más y menos seguros de Phoenix de cara a encontrar un buen lugar para poner un negocio.

<a id="id1"></a>

## 1. Criminalidad desde el año 2015 hasta el 2018

En primer lugar, cargamos el dataset de crímenes obtenido directamente de la página gubernamental de Phoenix. El proceso será realizarlo para todas las fechas que se tienen excepto 2019. El número de crímenes en el dataset esta diferenciado por fecha y por código postal, nosotros agruparemos por código postal para obtener el total de cada zona.

In [2]:
import pandas as pd
df_crime = pd.read_csv("Dataset/crime-data_crime-data_crimestat.csv")
df_crime = df_crime[['OCCURRED ON', 'UCR CRIME CATEGORY', 'ZIP']]
df_crime = df_crime.dropna()
df_crime['ZIP'] = df_crime['ZIP'].astype(int)

/home/javier/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_crime = df_crime.rename(columns={'OCCURRED ON': 'Fecha', 'UCR CRIME CATEGORY': 'Categoria'})

In [4]:
df_crime['Fecha'] = pd.to_datetime(df_crime['Fecha'])
df_crime['Fecha'] = df_crime['Fecha'].dt.strftime('%Y')
df_crime = df_crime[df_crime['Fecha'] != "2019"]
df_crime

,Fecha,Categoria,ZIP
0,2015,RAPE,85006
1,2015,LARCENY-THEFT,85014
2,2015,LARCENY-THEFT,85085
3,2015,MOTOR VEHICLE THEFT,85029
4,2015,MOTOR VEHICLE THEFT,85043
...,...,...,...
204537,2018,ARSON,85044
204538,2018,LARCENY-THEFT,85041
204539,2018,MOTOR VEHICLE THEFT,85035
204540,2018,BURGLARY,85021


In [5]:
df_crime =df_crime.groupby("ZIP").count().rename(columns={'Fecha':'count'})['count'].to_frame()
df_crime['codigo_postal'] = df_crime.index

<a id="id2"></a>

## 2. Graficación de las zonas en un mapa

Para ver cuales son las **zonas más seguras**, podríamos dejarlo en forma de tabla, solo es necesario ver cuales tienen menos crímenes. Sin embargo, **verlo de forma visual nos puede ayudar** a saber cuál esta cerca del centro, cuáles estan cerca unos de otros... etc.

En primer lugar obtenemos un **json que nos da información sobre los puntos de latitud y longitud de los límites de cada codigo postal de Arizona**, estado en el que se encuentra Phoenix. Con estos datos ya podemos graficar los crímenes por código postal.

In [6]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/az_arizona_zip_codes_geo.min.json') as response:
    data = json.load(response)
geozips = []
for i in range(len(data['features'])):
    data['features'][i]['id'] = data['features'][i]['properties'].pop('ZCTA5CE10')
    geozips.append(data['features'][i])

        
new_json = dict.fromkeys(['type', 'features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=new_json, locations=df_crime.codigo_postal, z=df_crime['count'],
                                    colorscale="Reds",
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=9, mapbox_center = {"lat": 33.547890, "lon": -112.067404})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Resultados
##### Ponemos las imágenes ya que es posible que los mapas interactivos no funcionen al reejecutar
Lo que vemos en el mapa son las zonas por código postal coloreadas según el número de crímenes, aparece la escala a la derecha:

<img src="https://i.imgur.com/oIo544j.png"><br>

Lo que nos interesa es ver cuál es la **zona más segura cerca del centro**, para encontrarlo nos fijaremos en la zona que menos delitos haya tenido en el tiempo. Entre ellos, el mejor de todos es es que aparece de color más claro y cerca del centro, con código postal 85012. Algo a tener en cuenta es que es posible que esta zona tenga menos población, menos calles, menos lugares donde poner negocios... etc. Por ello, sería más correcto ver estos datos de delitos con respecto a alguna otra variable.

<img src="https://i.imgur.com/mvBIW3T.png"><br>


Aunque lo ideal sería tener todo en cuenta, nosotros vamos a tener en cuenta la población de cada una de estas zonas:
Ha sido complicado encontrar un **conjunto de datos de la población de Phoenix** dividido por códigos postales y hemos encontrado **el más actualizado de 2016**, por lo que supondremos que los cambios entre dos años no sean muy grandes. Para ello **nos quedaremos con los datos de ese año tanto para la población como para los delitos**:

<a id="id3"></a>

## 3. Adición de la población de las zonas

De esta manera, podemos saber si la zona con poca cantidad de delitos podría ser porque haya menos población. Esto podría ser por muchos otros factores como hemos dicho antes (numero de negocios en la zona, pocas calles... etc), pero esta nos podría servir.

In [7]:
population = pd.read_csv("Dataset/pop-by-zip-code.csv")
population = population[['zip_code', 'y-2016']]
population

,zip_code,y-2016
0,601,17800
1,602,39716
2,603,51565
3,606,6320
4,610,27976
...,...,...
33115,99923,0
33116,99925,908
33117,99926,1667
33118,99927,81


Obtención de los datos de **delitos para el año 2016**:

In [8]:
df_crime = pd.read_csv("Dataset/crime-data_crime-data_crimestat.csv")
df_crime = df_crime[['OCCURRED ON', 'UCR CRIME CATEGORY', 'ZIP']]
df_crime = df_crime.dropna()
df_crime['ZIP'] = df_crime['ZIP'].astype(int)

In [9]:
df_crime = df_crime.rename(columns={'OCCURRED ON': 'Fecha', 'UCR CRIME CATEGORY': 'Categoria'})

In [10]:
df_crime['Fecha'] = pd.to_datetime(df_crime['Fecha'])
df_crime['Fecha'] = df_crime['Fecha'].dt.strftime('%Y')
df_crime = df_crime[df_crime['Fecha'] == "2016"]
df_crime

,Fecha,Categoria,ZIP
10171,2016,RAPE,85017
10172,2016,LARCENY-THEFT,85029
10173,2016,MOTOR VEHICLE THEFT,85041
10174,2016,RAPE,85051
10175,2016,LARCENY-THEFT,85053
...,...,...,...
73541,2016,ROBBERY,85006
73542,2016,RAPE,85031
73543,2016,AGGRAVATED ASSAULT,85042
73544,2016,ROBBERY,85008


In [11]:
df_crime =df_crime.groupby("ZIP").count().rename(columns={'Fecha':'count'})['count'].to_frame()
df_crime['zip_code'] = df_crime.index

<a id="id3.1"> </a>

### 3.1. Ponderación de delitos y población

La ponderación que usaremos tendrá en cuenta la población y el número de delitos por zona postal:

Ponderacion = Poblacion / Número de delitos. De esta forma, si en una zona hay mucha población y pocos delitos, el valor será alto, es decir, nos interesa esa zona. Si el valor es cercano a 0, hay tantos delitos como población en la zona, por lo que podría ser una zona más peligrosa.

In [12]:
df = pd.merge(df_crime, population, on='zip_code', how='inner')
df['ponderacion'] = df['y-2016']/df['count']
df

,count,zip_code,y-2016,ponderacion
0,655,85003,9079,13.861069
1,821,85004,5209,6.344702
2,1248,85006,27804,22.278846
3,965,85007,14461,14.985492
4,3034,85008,59490,19.607779
...,...,...,...,...
82,1,85381,25767,25767.000000
83,2,85382,39689,19844.500000
84,1,85383,42126,42126.000000
85,5,85392,38097,7619.400000


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=new_json, locations=df.zip_code, z=df['ponderacion'],
                                    colorscale="Jet",
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=9, mapbox_center = {"lat": 33.547890, "lon": -112.067404})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Vemos las **zonas con las diferentes ponderaciones** y podemos ver como toda la **parte central de Phoenix es exactamente igual**, al menos a esta escala. Para comprender el mapa, los colores azules (morados) son los cercanos al 0, es decir, zonas con números cercanos de delitos y de población, y las zonas más rojas o marrones, las zonas que tienen un gran número de habitantes pero bajo de delitos.

Es lógico pensar que **en las zonas más cercanas a la ciudad habrá más delitos** y **en las afueras habrá menos delitos**. La zona que más llama la atención es BUCKEYE. Incluso se puede ver en los mapas de antes sobre delitos que esta zona ni aparece diferente, ya que tiene tan pocos delitos que es muy clara.

<img src="https://i.imgur.com/asgV9OX.png"></br>

**Ahora vamos a mostrar solo las zonas con ponderación del mismo color (la zona azul céntrica) para centrarnos en estas zonas y poder ver diferencias.**

In [13]:
df = df[df['ponderacion'] < 1000]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=new_json, locations=df.zip_code, z=df['ponderacion'],
                                    colorscale="Jet",
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=9, mapbox_center = {"lat": 33.547890, "lon": -112.067404})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Podemos observar como aún bajando la escala, la zona sigue siendo prácticamente igual, a excepción de algunos lugares. En concreto, un lugar que aparece marrón/rojo a un lateral:

<img src="https://i.imgur.com/6aqRkXj.png"></br>

Este lugar es **Paradise Valley, un pueblo de Arizona** y que según aparentan las fotos de la zona, es un pueblo con zonas de lujo. Es un lugar realmente interesante ya que tiene un número de delitos muy bajo para la población que tiene. *Cabe destacar que este lugar aparece en algunas páginas web como un lugar donde retirarse cuando eres rico.*
<img src="https://i.imgur.com/r0q8Zm3.png"></br>

<a id="id4"></a>

## 4. ¿Podemos sacar provecho a estos datos?

Ya que hemos encontrado un lugar en la zona que parece realmente seguro, podríamos ver si hay algún negocio que sea típico en otros lugares y que no exista en el pueblo de Paradise Valley.

Para ello compararemos la cantidad total de negocios de todo Phoenix dividido por categorías y lo compararemos en proporción (viendo cuáles son los más típicos en ambos lugares) con el del pueblo.

In [14]:
df_negocios = pd.read_csv("Dataset/datos_negocios_limpios.csv")
phoenix = df_negocios[(df_negocios['ciudad'] == "Phoenix") & (df_negocios['abierto'] == 1)]

In [15]:
import ast

categoriasBarrioMalo = []

aux = phoenix
#aux['categorias'].apply(lambda x: print(x))

aux['categorias'].apply(lambda x: categoriasBarrioMalo.extend(ast.literal_eval(x.replace(" ", ""))))
categorias = pd.DataFrame(categoriasBarrioMalo).rename(columns={0:"Categoria"})
categorias['count'] = 0
categorias.groupby('Categoria').count().sort_values('count', ascending=False).head(5)

,count
Categoria,
HomeServices,2905
Restaurants,2719
Shopping,2547
Health&Medical,1985
LocalServices,1751


In [16]:
import ast

categoriasBarrioMalo = []

aux = phoenix[phoenix['codigo_postal'] == "85253"]
#aux['categorias'].apply(lambda x: print(x))

aux['categorias'].apply(lambda x: categoriasBarrioMalo.extend(ast.literal_eval(x.replace(" ", ""))))
categorias = pd.DataFrame(categoriasBarrioMalo).rename(columns={0:"Categoria"})
categorias['count'] = 0
categorias.groupby('Categoria').count().sort_values('count', ascending=False).head(5)

,count
Categoria,
Health&Medical,2
HomeServices,2
LocalServices,2
Beauty&Spas,1
MedicalSpas,1


Mirando las tablas anteriores podemos observar a ojo como en el pueblo **hay poca cantidad de negocios**, tal vez porque sea un pueblo y no una ciudad, pero resulta extraño ya que es de gran extensión y además, según lo que se ha visto en fotos, un **lugar de mucho lujo**.

Se podría aprovechar esta oportunidad e intentar abrir un negocio de cualquier tipo que se adapte a las necesidades de este tipo de población, pero de poder elegir, **nosotros diríamos de abrir un restaurante o una tienda en las zonas de más prestigio**, atendiendo únicamente a lo anterior y por intuición.

**Esta parte ha sido un añadido sin mucho análisis y solo como complemento de lo que se podría hacer, esta claro que para saber que tipo de negocio abrir habria que tener en cuenta muchas otras cosas, como la renta per camita, la ubicación exacta de los negocios actuales... etc.**